In [1]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from bs4 import BeautifulSoup
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import albumentations
import albumentations.pytorch
from matplotlib import pyplot as plt
import os
import random
import utility
import rasterio
import wandb
import tqdm
import shutil
import pandas as pd

In [2]:
from utility import ShipDataset
from utility import ShipDataset_YOLO

In [3]:
# if matplot doesnt work
import matplotlib
#matplotlib.use('TkAgg')

# same seed
torch.manual_seed(0)

# garbage collection
import gc
gc.collect()
torch.cuda.empty_cache()

# CUDA
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
batch_size = 1

#### Faster R-CNN and RetinaNet Datasets and Loaders

In [8]:
val_dataset = ShipDataset(
    path = 'test_yolo/',
    #transform=transform
)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(val_dataset)

736

In [18]:
inshore_dataset = ShipDataset(
    path = 'in',
    #transform=transform
)

inshore_loader = torch.utils.data.DataLoader(inshore_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(inshore_dataset)

117

In [19]:
offshore_dataset = ShipDataset(
    path = 'off',
    #transform=transform
)

offshore_loader = torch.utils.data.DataLoader(offshore_dataset, batch_size=batch_size,collate_fn=utility.collate_fn)
len(offshore_dataset)

619

#### YOLO Data

In [ ]:
val_dataset_yolo = ShipDataset_YOLO(
    path = 'test_yolo/',
    #transform=transform
)

val_loader_yolo = torch.utils.data.DataLoader(val_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(val_loader_yolo)

In [ ]:
inshore_dataset_yolo = ShipDataset_YOLO(
    path = 'in',
    #transform=transform
)

inshore_loader_yolo = torch.utils.data.DataLoader(inshore_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(inshore_dataset_yolo)

In [ ]:
offshore_dataset_yolo = ShipDataset_YOLO(
    path = 'off',
    #transform=transform
)

offshore_loader_yolo = torch.utils.data.DataLoader(offshore_dataset_yolo, batch_size=batch_size,collate_fn=utility.collate_fn)
len(offshore_dataset_yolo)

### RetinaNet Results

In [5]:
retina = torchvision.models.detection.retinanet_resnet50_fpn(num_classes=2)
retina.load_state_dict(torch.load("models/retina300R2.pt"))
retina.eval()
retina.to(device)
print()

In [12]:
retina_thresholds = [0.4,0.5,0.6,0.7,0.8,0.9,0.95]
retina_results = []
for t in retina_thresholds:
    temp = utility.get_metric(retina,val_loader,t,device)
    retina_results.append(temp)

/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/alex_notebooks/utility.py:465: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))


Precision: 0.457 
 Recall: 0.873 
 AP: 0.739 
 F1: 0.600 
 mAP 0.739 
 time: 52.603 
 threshold: 0.4
Precision: 0.524 
 Recall: 0.859 
 AP: 0.731 
 F1: 0.651 
 mAP 0.731 
 time: 52.652 
 threshold: 0.5
Precision: 0.584 
 Recall: 0.839 
 AP: 0.720 
 F1: 0.689 
 mAP 0.720 
 time: 52.691 
 threshold: 0.6
Precision: 0.635 
 Recall: 0.812 
 AP: 0.704 
 F1: 0.713 
 mAP 0.704 
 time: 52.566 
 threshold: 0.7
Precision: 0.685 
 Recall: 0.774 
 AP: 0.679 
 F1: 0.727 
 mAP 0.679 
 time: 52.624 
 threshold: 0.8
Precision: 0.756 
 Recall: 0.701 
 AP: 0.627 
 F1: 0.728 
 mAP 0.627 
 time: 52.368 
 threshold: 0.9
Precision: 0.801 
 Recall: 0.616 
 AP: 0.560 
 F1: 0.696 
 mAP 0.560 
 time: 52.610 
 threshold: 0.95


##### Retina Test Results

In [13]:
cols = ["Precision", "Recall", "AP" , "f1" ,"mAP", "Time (seconds)", "Threshold"]
retina_results_df = pd.DataFrame(retina_results, columns=cols)
retina_results_df

,Precision,Recall,AP,f1,mAP,Time (seconds),Threshold
0,0.456967,0.873003,0.738501,0.599913,0.738501,52.602701,0.40
1,0.523590,0.858705,0.731477,0.650526,0.731477,52.651508,0.50
2,0.584066,0.838520,0.720244,0.688536,0.720244,52.691252,0.60
3,0.634780,0.812027,0.704039,0.712546,0.704039,52.565931,0.70
4,0.684896,0.774180,0.679099,0.726806,0.679099,52.623981,0.80
5,0.756120,0.701430,0.626657,0.727749,0.626657,52.367927,0.90
6,0.801314,0.615643,0.559584,0.696314,0.559584,52.610323,0.95


In [17]:
precision = retina_results_df['Precision']
recall = retina_results_df['Recall']
# Create plot
plt.plot(recall, precision)
# Add axis labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision vs Recall RetinaNet')
# Show the plot
plt.show()

In [20]:
# from Precision Recall, best threhold
retina_threshold = 0.7

##### Retina Test Inshore Results

In [23]:
retina_results_in = []
temp = utility.get_metric(retina,inshore_loader,retina_threshold,device)
retina_results_in.append(temp)
retina_results_in_df = pd.DataFrame(retina_results_in, columns=cols)
retina_results_in_df

Precision: 0.367 
 Recall: 0.624 
 AP: 0.363 
 F1: 0.462 
 mAP 0.363 
 time: 8.924 
 threshold: 0.7


,Precision,Recall,AP,f1,mAP,Time (seconds),Threshold
0,0.3666,0.624009,0.362535,0.461861,0.362535,8.923793,0.7


##### Retina Test Offshore Results

In [24]:
retina_results_off = []
temp = utility.get_metric(retina,offshore_loader,retina_threshold,device)
retina_results_off.append(temp)
retina_results_off_df = pd.DataFrame(retina_results_off, columns=cols)
retina_results_off_df

Precision: 0.897 
 Recall: 0.923 
 AP: 0.886 
 F1: 0.910 
 mAP 0.886 
 time: 43.792 
 threshold: 0.7


/home/alex/school/DSC180A/SAR-satelite-image-ship-detection/alex_notebooks/utility.py:465: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mAP = torch.mean(torch.tensor(AP,dtype=torch.float64))


,Precision,Recall,AP,f1,mAP,Time (seconds),Threshold
0,0.896686,0.923077,0.885786,0.90969,0.885786,43.79236,0.7


### Faster R-CNN Results